In [65]:
import random
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForCausalLM
from datasets import load_dataset, DatasetDict
import torch

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load data

In [4]:
polemo = load_dataset("clarin-pl/polemo2-official")
polemo_n_cls = len(polemo['train'].features['target'].names)

### BERT

In [35]:
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
bert = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased", num_labels=polemo_n_cls, trust_remote_code=True)
bert.to(device)
bert.eval();

Augment with [MASK] token

In [38]:
def augment_text_with_masking(examples, mask_probability=0.15, top_k=3):
    augmented_texts = []
    for text in examples['text']:
        # tokenize text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(bert.device) for k, v in inputs.items()}
        input_ids = inputs["input_ids"]
        
        # mask random tokens
        mask_positions = [
            i for i in range(1, input_ids.size(1) - 1)  # exclude CLS and SEP tokens
            if random.random() < mask_probability
        ]
        
        # skip augmentation if no token was selected
        if not mask_positions:
            augmented_texts.append(text)
            continue
        
        # clone and mask selected ids
        input_ids_masked = input_ids.clone()
        for pos in mask_positions:
            input_ids_masked[0, pos] = tokenizer.mask_token_id
        
        # infer masked tokens
        with torch.no_grad():
            outputs = bert(input_ids_masked)
            predictions = outputs.logits
        
        # replace masked tokens with top predicted tokens
        for pos in mask_positions:
            top_tokens = torch.topk(predictions[0, pos], top_k).indices
            selected_token = top_tokens[random.randint(0, top_k - 1)]
            input_ids[0, pos] = selected_token
        
        # decode augmented text
        augmented_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        augmented_texts.append(augmented_text)
    
    return {"text": augmented_texts, "target": examples["target"]}

In [ ]:
augmented_data_seq = polemo.map(
    augment_text_with_masking, 
    batched=True, 
    batch_size=64, 
    desc="Augmenting data with masking", 
    fn_kwargs={"mask_probability": 0.3, "top_k": 3}
)

In [62]:
for idx in random.sample(range(polemo["train"].num_rows), 4):
    example = polemo['train'][idx]
    print(f"Original text:\n{example['text']}\n")
    
    augmented_example = augmented_data_seq['train'][idx]
    print(f"Augmented text:\n{augmented_example['text']}\n\n")

Original text:
Zakochali śmy się z mężem w hotelach PURO . Byli śmy oczarowani Puro Hotel Gdańsk . Jak tylko nadarzyła się okazja spędzenia weekendu " gdzieś " to bez wahania wybrali śmy takie miasto gdzie jest Puro Hotel . Padło na Poznań . I znów rewelacja , nie zawiedli śmy się na niczym . Tak jak oczekiwali śmy : pokoje , jedzenie , lobby , komfort , czystość , obsługa , wygoda , lokalizacja , wszystko na najwyższym poziomie . Teraz czeka nas Puro Hotel Kraków . Do zobaczenia ! Zachęcamy !

Augmented text:
Zakochali śmy się z mężem w hotelach P& O . Byli śmy zaczarż Puro Hotel Gdańsk . Jak tylko narzała się okazja spędzenia weekendu " gdzieś " to bez wahania wybrali śmy takie miasto gdzie jest Puro Hotel GPadka na weekend . I znów rewelacja , nie zawieliśmy się z niczym . Tak jak chcieli śmy : pokoje , wy, kuchnia , komfort , czystość , splwycja , ne wszystko na najwyższym poziomie , Teraz czeka nuro uro Hotel Kraków . o zobaczenia . Zachęcamy .


Original text:
Test bazy 2wayFix +

In [49]:
augmented_data_seq.save_to_disk("data/polemo2-augmented-BERT")

Saving the dataset (0/1 shards):   0%|          | 0/6573 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/823 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/820 [00:00<?, ? examples/s]

### GPT2

In [63]:
tokenizer = AutoTokenizer.from_pretrained("sdadas/polish-gpt2-medium")
gpt2 = AutoModelForCausalLM.from_pretrained("sdadas/polish-gpt2-medium", num_labels=polemo_n_cls, trust_remote_code=True)
gpt2.to(device)
gpt2.eval();

Augment with generative model

In [64]:
def augment_text_with_generation(examples, temperature=0.7, top_k=3, top_p=0.9, max_new_tokens=50):
    augmented_texts = []
    for text in examples['text']:
        # tokenize text
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
        inputs = {k: v.to(gpt2.device) for k, v in inputs.items()}
        
        # Generate new samples
        output = gpt2.generate(
            inputs["input_ids"],
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        # Decode generated text and add it to augmented texts
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        augmented_texts.append(generated_text)
    
    return {"text": augmented_texts, "target": examples["target"]}

In [67]:
subset_polemo = DatasetDict({
    "train": polemo["train"].select(range(polemo["train"].num_rows//3)),
    "validation": polemo["validation"].select(range(polemo["validation"].num_rows//3)),
    "test": polemo["test"].select(range(polemo["test"].num_rows//3)),
})

In [69]:
augmented_data_seq = subset_polemo.map(
    augment_text_with_generation,
    batched=True,
    batch_size=32,
    desc="Augmenting data with generative model",
    fn_kwargs={"temperature": 0.7, "top_k": 3, "top_p": 0.8, "max_new_tokens": 50},
)

Augmenting data with generative model:   0%|          | 0/2191 [00:00<?, ? examples/s]

Augmenting data with generative model:   0%|          | 0/274 [00:00<?, ? examples/s]

Augmenting data with generative model:   0%|          | 0/273 [00:00<?, ? examples/s]

In [72]:
for idx in random.sample(range(subset_polemo["train"].num_rows), 4):
    example = subset_polemo['train'][idx]
    print(f"Original text:\n{example['text']}\n")
    
    augmented_example = augmented_data_seq['train'][idx]
    print(f"Augmented text:\n{augmented_example['text']}\n\n")

Original text:
Do Dr skierował mnie lekarz z CZMP , prywatnie wykonywał mi USG i niestety z przeprowadzonego przez niego badania USG nie jestem zadowolona . Nie raczył mi wytłumaczyć w czasie badania zauważonej zmiany , tylko dyktował asystentce . Brak w jego gabinecie przebieralni , co jest niezbędne przy przygotowaniu do tego badania . Nie polecam . liczy się tylko kasa , kasa .

Augmented text:
Do Dr skierował mnie lekarz z CZMP , prywatnie wykonywał mi USG i niestety z przeprowadzonego przez niego badania USG nie jestem zadowolona . Nie raczył mi wytłumaczyć w czasie badania zauważonej zmiany , tylko dyktował asystentce . Brak w jego gabinecie przebieralni , co jest niezbędne przy przygotowaniu do tego badania . Nie polecam . liczy się tylko kasa , kasa .


Original text:
Hotel może na jedną noc , ale nie więcej ! Ja musiała m niestety spędzić tam 11 nocy i mimo , że zabukowany był na 12 , to ostatnią noc wolała m spędzić w samochodzie niż spać w tym hotelu . Zaznaczam , że sypiam 

In [70]:
augmented_data_seq.save_to_disk("data/polemo2-augmented-GPT2")

Saving the dataset (0/1 shards):   0%|          | 0/2191 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/274 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/273 [00:00<?, ? examples/s]